<a href="https://colab.research.google.com/github/yufengg/advent-of-code-2024/blob/master/AoC_2024_day_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: python code download advent of code file

import requests
import os
from datetime import datetime

def download_aoc_input(year, day, session_cookie):
  """Downloads the input file for a specific Advent of Code day.

  Args:
    year: The year of the challenge (e.g., 2023).
    day: The day of the challenge (e.g., 1).
    session_cookie: Your session cookie from the Advent of Code website.
  """

  url = f"https://adventofcode.com/{year}/day/{day}/input"
  headers = {
      "User-Agent": "github.com/yourusername/yourrepo (or your email)",  # Replace with your info
      "Cookie": f"session={session_cookie}"
  }

  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Create a directory for the year if it doesn't exist
    os.makedirs(str(year), exist_ok=True)

    filename = os.path.join(str(year), f"day{day}.txt")
    with open(filename, "w") as file:
      file.write(response.text)
    print(f"Successfully downloaded input for year {year}, day {day} to {filename}")
    return filename

  except requests.exceptions.RequestException as e:
    print(f"Error downloading input: {e}")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")


# Example usage:  Replace with your actual values
year = 2024  #@param {type:"integer"}
day = 8 #@param {type:"integer"}
session_cookie = "53616c7465645f5fd6809f86933785dcec3c4f79b521758a796b0808898e32ff9a09d06dd15282b2a68340b7bf970ab66894fa0bf9271caa521bc5d5fc8df3b9" #@param {type:"string"}

input_filename = download_aoc_input(year, day, session_cookie)

Successfully downloaded input for year 2024, day 8 to 2024/day8.txt


In [3]:
import jax
import jax.numpy as jnp
import numpy as np


In [32]:

def read_input(filename, parser=None):
  with open(filename, 'r') as file:
    lines = file.read().splitlines()
    if parser:
      lines = list(map(parser, lines))
  return lines


In [33]:
input_filename = f'{year}/day{day}.txt'
def parser(line):

  return line

input_lines = read_input(input_filename, parser)

In [34]:
input_lines[0:5]

['...d............................J.................',
 '......e.............................J.............',
 '..........6............7..........................',
 '........................P7........................',
 '..................................................']

# TEST data

In [35]:
test_data = """............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............
"""

test_filename = f'{year}/test.txt'

with open(test_filename, 'w') as file:
  file.write(test_data)

In [36]:
test_lines = read_input(test_filename, parser)


In [37]:
test_lines

['............',
 '........0...',
 '.....0......',
 '.......0....',
 '....0.......',
 '......A.....',
 '............',
 '............',
 '........A...',
 '.........A..',
 '............',
 '............']

# Part 1

find all instances of each letter/number

for each letter, compute antinode locations based on every single unique pairing of nodes of that letter.

  how to find the various pairings for a set of the same letters? SUM[n-1...1]
  Start from first and pair with the next, go down the array to the end; then start from 2nd, go down the list, etc. until reach the 2nd to last value.

Check if it'll be on the map or not, keep the valid ones.

distance calculation is a delta_xy in both directions

So if the 2 points are x1 = [5,7] and x2 = [9,10], the gap is x2-x1 = [4, 3]

* [5,7] - [4, 3] = [1, 4] , aka x1-delta
* [9,10] + [4, 3] = [13, 13], aka x2+delta

In [91]:
# class to store the grid, boundary validation function, delta calculator,

class Grid():
  def __init__(self, input_lines):
    self.grid = input_lines
    self.rows = len(input_lines)
    self.cols = len(input_lines[0])
    self.antinodes = set()
    self.towers = {} # { letter: list[ location_tuple ] }
    self.find_towers()

  def in_bounds(self, pos):
    x,y = pos
    return 0 <= x < self.rows and 0 <= y < self.cols

  def find_towers(self):
    for row in range(self.rows):
      for col in range(self.cols):
        loc_val = self.grid[row][col]
        if loc_val != '.':
          # if self.grid[row][col] not in self.towers:
          #   self.towers[self.grid[row][col]] = []
          location_list = self.towers.get(loc_val, [])
          location_list.append((row, col))
          self.towers[loc_val] = location_list

  def delta(self, x1, y1, x2, y2):
    return (x2-x1, y2-y1)

  def get_antinode(self, x1, y1, x2, y2):
    delta_x, delta_y = self.delta(x1, y1, x2, y2)
    return (x1-delta_x, y1-delta_y), (x2+delta_x, y2+delta_y)

  def tower_combos(self, tower_letter):
    tower_locations = self.towers[tower_letter]
    combos = []
    for i in range(len(tower_locations)-1):
      for j in range(i+1, len(tower_locations)):
        # print(i,j)
        combos.append((tower_locations[i], tower_locations[j]))
    return combos

  def find_antinodes(self):
    for tower_letter, tower_locations in self.towers.items():
      for (x1,y1),(x2, y2) in self.tower_combos(tower_letter):
        a, b = self.get_antinode(x1,y1,x2,y2)
        if self.in_bounds(a):
          self.antinodes.add(a)
        if self.in_bounds(b):
          self.antinodes.add(b)
    return self.antinodes


In [92]:
g = Grid(test_lines)

In [93]:
g.towers

{'0': [(1, 8), (2, 5), (3, 7), (4, 4)], 'A': [(5, 6), (8, 8), (9, 9)]}

In [74]:
g.tower_combos('A')

[((5, 6), (8, 8)), ((5, 6), (9, 9)), ((8, 8), (9, 9))]

In [75]:
(x1,y1),(x2, y2) = g.tower_combos('0')[4]

In [76]:
(x1,y1),(x2, y2)

((2, 5), (4, 4))

In [77]:
(a1, b1), (a2, b2) = g.get_antinode(x1,y1,x2,y2)
(a1, b1), (a2, b2)

((0, 6), (6, 3))

In [86]:
g.in_bounds((a1,b1)) , g.in_bounds((a2,b2))

(True, True)

In [87]:
antinodes = g.get_antinode(x1,y1,x2,y2)

In [88]:
list(map(g.in_bounds, antinodes))

[True, True]

In [94]:
an = g.find_antinodes()

In [97]:
an


{(0, 6),
 (0, 11),
 (1, 3),
 (2, 4),
 (2, 10),
 (3, 2),
 (4, 9),
 (5, 1),
 (5, 6),
 (6, 3),
 (7, 0),
 (7, 7),
 (10, 10),
 (11, 10)}

In [98]:
g = Grid(input_lines)
an = g.find_antinodes()

len(an)

291

# Part 2

instead of creating 2 fixed antinodes, we now have antinodes propogating outwards as many times as needed to leave the map. Additionally, the antenna itself counts as a node, assuming there are at least 2 total.



In [131]:
# class to store the grid, boundary validation function, delta calculator,

class Grid():
  def __init__(self, input_lines):
    self.grid = input_lines
    self.rows = len(input_lines)
    self.cols = len(input_lines[0])
    self.antinodes = set()
    self.towers = {} # { letter: list[ location_tuple ] }
    self.find_towers()

  def in_bounds(self, pos):
    x,y = pos
    return 0 <= x < self.rows and 0 <= y < self.cols

  def find_towers(self):
    for row in range(self.rows):
      for col in range(self.cols):
        loc_val = self.grid[row][col]
        if loc_val != '.':
          # if self.grid[row][col] not in self.towers:
          #   self.towers[self.grid[row][col]] = []
          location_list = self.towers.get(loc_val, [])
          location_list.append((row, col))
          self.towers[loc_val] = location_list

  def delta(self, x1, y1, x2, y2):
    return (x2-x1, y2-y1)

  def get_antinode(self, x1, y1, x2, y2):
    delta_x, delta_y = self.delta(x1, y1, x2, y2)
    return (x1-delta_x, y1-delta_y), (x2+delta_x, y2+delta_y)

  def get_antinode_list(self, x1, y1, x2, y2):
    delta_x, delta_y = self.delta(x1, y1, x2, y2)
    # start with self and first order harmonics
    # x1_harmonic = (x1-delta_x, y1-delta_y)
    # x2_harmonic = (x2+delta_x, y2+delta_y)
    # antinodes = [(x1, y1), (x2, y2)]
    #  x1_harmonic, x2_harmonic]

    self.harmonize((x1,y1), (delta_x, delta_y), -1)
    self.harmonize((x2,y2), (delta_x, delta_y), 1)

    # test for more


  # direction is -1 or 1
  def harmonize(self, initial, delta, direction):
    print(f'harmonizing from {initial}, delta: {delta}, dir: {direction}')
    x1, y1 = initial
    x2, y2 = delta
    x2, y2 = x2*direction, y2*direction

    while self.in_bounds((x1,y1)):
      print(f'adding {x1,y1}')
      self.antinodes.add((x1,y1))
      x1, y1 = x1+x2, y1+y2 # move 1 delta jump

    return

  def tower_combos(self, tower_letter):
    tower_locations = self.towers[tower_letter]
    combos = []
    for i in range(len(tower_locations)-1):
      for j in range(i+1, len(tower_locations)):
        # print(i,j)
        combos.append((tower_locations[i], tower_locations[j]))
    return combos

  def find_antinodes(self):
    for tower_letter, tower_locations in self.towers.items():
      for (x1,y1),(x2, y2) in self.tower_combos(tower_letter):
        self.get_antinode_list( x1, y1, x2, y2)

        # a, b = self.get_antinode(x1,y1,x2,y2)
        # if self.in_bounds(a):
        #   self.antinodes.add(a)
        # if self.in_bounds(b):
        #   self.antinodes.add(b)
    return self.antinodes


In [135]:
g = Grid(input_lines)


In [136]:
g.find_antinodes()

harmonizing from (0, 3), delta: (7, 12), dir: -1
adding (0, 3)
harmonizing from (7, 15), delta: (7, 12), dir: 1
adding (7, 15)
adding (14, 27)
adding (21, 39)
harmonizing from (0, 3), delta: (10, 7), dir: -1
adding (0, 3)
harmonizing from (10, 10), delta: (10, 7), dir: 1
adding (10, 10)
adding (20, 17)
adding (30, 24)
adding (40, 31)
harmonizing from (0, 3), delta: (17, 1), dir: -1
adding (0, 3)
harmonizing from (17, 4), delta: (17, 1), dir: 1
adding (17, 4)
adding (34, 5)
harmonizing from (7, 15), delta: (3, -5), dir: -1
adding (7, 15)
adding (4, 20)
adding (1, 25)
harmonizing from (10, 10), delta: (3, -5), dir: 1
adding (10, 10)
adding (13, 5)
adding (16, 0)
harmonizing from (7, 15), delta: (10, -11), dir: -1
adding (7, 15)
harmonizing from (17, 4), delta: (10, -11), dir: 1
adding (17, 4)
harmonizing from (10, 10), delta: (7, -6), dir: -1
adding (10, 10)
adding (3, 16)
harmonizing from (17, 4), delta: (7, -6), dir: 1
adding (17, 4)
harmonizing from (0, 32), delta: (1, 4), dir: -1
add

{(26, 30),
 (18, 26),
 (18, 35),
 (7, 35),
 (27, 13),
 (29, 32),
 (8, 9),
 (30, 9),
 (29, 41),
 (30, 18),
 (21, 37),
 (19, 18),
 (6, 48),
 (11, 14),
 (21, 46),
 (41, 24),
 (33, 20),
 (33, 29),
 (4, 2),
 (45, 3),
 (44, 47),
 (34, 12),
 (45, 12),
 (25, 43),
 (3, 6),
 (45, 21),
 (14, 15),
 (3, 15),
 (3, 24),
 (14, 24),
 (22, 37),
 (46, 4),
 (14, 33),
 (3, 33),
 (22, 46),
 (3, 42),
 (26, 7),
 (38, 9),
 (7, 3),
 (49, 18),
 (15, 16),
 (47, 48),
 (36, 48),
 (26, 25),
 (18, 21),
 (48, 22),
 (40, 27),
 (48, 40),
 (8, 4),
 (6, 34),
 (30, 13),
 (21, 32),
 (11, 9),
 (41, 10),
 (10, 22),
 (33, 24),
 (41, 37),
 (25, 20),
 (25, 29),
 (45, 7),
 (34, 7),
 (2, 36),
 (34, 16),
 (45, 16),
 (25, 47),
 (3, 10),
 (34, 25),
 (22, 23),
 (45, 25),
 (3, 19),
 (14, 28),
 (3, 37),
 (26, 2),
 (49, 4),
 (14, 37),
 (47, 34),
 (36, 34),
 (7, 7),
 (18, 7),
 (26, 20),
 (47, 43),
 (36, 43),
 (7, 16),
 (48, 17),
 (28, 48),
 (29, 13),
 (17, 48),
 (48, 26),
 (40, 31),
 (41, 5),
 (21, 36),
 (44, 1),
 (33, 1),
 (41, 14),
 (21

In [137]:
len(_)

1015

In [129]:
test_answer = """##....#....#
.#.#....0...
..#.#0....#.
..##...0....
....0....#..
.#...#A....#
...#..#.....
#....#.#....
..#.....A...
....#....A..
.#........#.
...#......##"""

In [130]:
for i,l in enumerate(test_answer.splitlines()):
  print(l)
  for j,c in enumerate(l):
    if c in '#A0':
      print(  (i,j))

##....#....#
(0, 0)
(0, 1)
(0, 6)
(0, 11)
.#.#....0...
(1, 1)
(1, 3)
(1, 8)
..#.#0....#.
(2, 2)
(2, 4)
(2, 5)
(2, 10)
..##...0....
(3, 2)
(3, 3)
(3, 7)
....0....#..
(4, 4)
(4, 9)
.#...#A....#
(5, 1)
(5, 5)
(5, 6)
(5, 11)
...#..#.....
(6, 3)
(6, 6)
#....#.#....
(7, 0)
(7, 5)
(7, 7)
..#.....A...
(8, 2)
(8, 8)
....#....A..
(9, 4)
(9, 9)
.#........#.
(10, 1)
(10, 10)
...#......##
(11, 3)
(11, 10)
(11, 11)


In [126]:
simple="""T.........
...T......
.T........
..........
..........
..........
..........
..........
..........
..........""".splitlines()

In [127]:
g = Grid(simple)
g.find_antinodes()

harmonizing from (0, 1), delta: (1, 3), dir: 1
adding (0, 1)
adding (1, 4)
adding (2, 7)
harmonizing from (0, 1), delta: (1, 3), dir: -1
adding (0, 1)
harmonizing from (0, 2), delta: (2, 1), dir: 1
adding (0, 2)
adding (2, 3)
adding (4, 4)
adding (6, 5)
adding (8, 6)
harmonizing from (0, 2), delta: (2, 1), dir: -1
adding (0, 2)
harmonizing from (1, 2), delta: (1, -2), dir: 1
adding (1, 2)
adding (2, 0)
harmonizing from (1, 2), delta: (1, -2), dir: -1
adding (1, 2)
adding (0, 4)


{(0, 1),
 (0, 2),
 (0, 4),
 (1, 2),
 (1, 4),
 (2, 0),
 (2, 3),
 (2, 7),
 (4, 4),
 (6, 5),
 (8, 6)}

In [128]:
print("""T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........""")

T....#....
...T......
.T....#...
.........#
..#.......
..........
...#......
..........
....#.....
..........


In [124]:
g.antinodes

{(0, 1),
 (0, 2),
 (0, 4),
 (1, 2),
 (1, 4),
 (2, 0),
 (2, 3),
 (2, 7),
 (4, 4),
 (6, 5),
 (8, 6)}

In [ ]:
456567238446228 - 456565678667482
# this was the diff between whether it early terminates, and the rule that broke the old code using that 1 example.
# Needed to check that the value matched the goal AND there were no more numbers in the queue.

1559778746